In [3]:
tickers = ['MGLU3.SA', 'VALE3.SA']
periodo = '1y'  # Opções: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Baixando Cotações e Calculando Retornos

In [1]:
!pip install -q yfinance

     |████████████████████████████████| 6.3MB 7.8MB/s 


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from ipywidgets import interact

In [4]:
def fix_col_names(df):
  return ['IBOV' if col =='^BVSP' else col.rstrip('.SA') for col in df.columns]

In [5]:
prices = yf.download(tickers, period=periodo, rounding=True )['Adj Close']
prices.columns = fix_col_names(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.to_list()

[*********************100%***********************]  2 of 2 completed


In [6]:
prices.tail()

,MGLU3,VALE3
Date,,
2021-05-12,18.59,114.33
2021-05-13,19.13,112.49
2021-05-14,19.15,110.56
2021-05-17,18.86,113.46
2021-05-18,18.97,114.60


In [7]:
retorno.tail()

,MGLU3,VALE3
Date,,
2021-05-12,-0.037785,-0.036978
2021-05-13,0.029048,-0.016094
2021-05-14,0.001045,-0.017157
2021-05-17,-0.015144,0.026230
2021-05-18,0.005832,0.010048


## 1. Criando a Carteira

In [8]:
def calcula_carteira(df, w1):
  pesos = [w1,(1-w1)]  
  df2 = df.dot(pesos).copy()
  return  df2.mean() * 252, df2.std() * np.sqrt(252)

In [9]:
# Criando 100 carteiras de dois ativos com pesos variáveis
carteira = pd.DataFrame()
for i in np.linspace(0,1, 101):
  media, std = calcula_carteira(retorno,i)
  carteira.at[i,'retorno'] = media
  carteira.at[i,'volatilidade'] = std

carteira

,retorno,volatilidade
0.00,0.968518,0.335859
0.01,0.962908,0.332398
0.02,0.957298,0.328989
0.03,0.951687,0.325634
0.04,0.946077,0.322334
...,...,...
0.96,0.429940,0.395891
0.97,0.424330,0.400020
0.98,0.418720,0.404179
0.99,0.413110,0.408367


In [14]:
# Carteira de volatilidade mínima
min_vol_idx = carteira['volatilidade'].idxmin()

In [13]:
# Calcula a fronteira eficiente
if (carteira['retorno'].iloc[0] > carteira['retorno'].iloc[-1]):
  fe = carteira.loc[:min_vol_idx, :]
else:
  fe = carteira.loc[min_vol_idx:, :]

## 2. Gráfico

In [15]:
def gerar_grafico(w1, mostrar_curvas):
  fig = go.Figure()

  # Desenha um ponto com o retorno e a volatilidade da carteira
  fig.add_scatter(y=[carteira.iloc[w1]['retorno']], 
                  x=[carteira.iloc[w1]['volatilidade']], 
                  text=['CARTEIRA'],
                  mode='markers+text', name='CARTEIRA')
  
  # Desenha os pontos das ações individuais
  fig.add_scatter(y=carteira.iloc[[-1,0]]['retorno'], 
                  x=carteira.iloc[[-1,0]]['volatilidade'], 
                  text=ativos,
                  mode='markers+text', name='Ações')
  
  # Desenha a curva
  fig.add_scatter(y=carteira['retorno'], 
                  x=carteira['volatilidade'],                  
                  mode='lines', name='Curva', 
                  visible=mostrar_curvas)

  # Plota carteira de volatilidade mínima   
  fig.add_scatter(y=[carteira.loc[min_vol_idx]['retorno']], 
                  x=[carteira.loc[min_vol_idx]['volatilidade']],                  
                  mode='markers', 
                  name='Carteira de Mínima Variância',
                  visible=mostrar_curvas)
      
  # Desenha a fronteira eficiente
  fig.add_scatter(y=fe['retorno'], 
                  x=fe['volatilidade'],                  
                  mode='lines', name='Fronteira Eficiente', 
                  line={'color':'red'},
                  visible=mostrar_curvas)
                
  fig.update_traces(textfont_size=12, 
                  textposition='top center', 
                  textfont_color='white',
                  hovertemplate='<b>retorno: </b> %{y:.1%}'+
                                '<br><b>volatilidade:</b> %{x:.1%}')
  fig.layout.autosize = False
  fig.layout.xaxis.title='Volatilidade'
  fig.layout.yaxis.title='Retorno Esperado'
  fig.layout.xaxis.range = [carteira['volatilidade'].min()-0.05, carteira['volatilidade'].max()+0.05]
  fig.layout.yaxis.range = [carteira['retorno'].min()-0.05, carteira['retorno'].max()+0.05]
  fig.layout.xaxis.tickformat = '.0%'
  fig.layout.yaxis.tickformat = '.0%'
  fig.layout.title = f"<b>{ativos[0]}:</b> {w1}%  <b>{ativos[1]}:</b> {100-w1}%"
  fig.layout.template = 'plotly_dark'
                   
  fig.show(config=dict(
                    displayModeBar=True
                ))

interact(gerar_grafico, w1=(0,100, 1), mostrar_curvas=False);

interactive(children=(IntSlider(value=50, description='w1'), Checkbox(value=False, description='mostrar_curvas…

## 3. Formulação Matemática

### Para uma portfólio com 2 Ativos

Sejam $w_1$ e $w_2$ os pesos e $R_1$ e $R_2$ os retornos esperados dos ativos 1 e 2 respectavemente.

$$R_P = w_1R_1 + w_2R_2$$

A **variância** $\sigma_p^2$ do portfólio é dada por:

$$ 
\sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\rho_{1,2}\sigma_1\sigma_2
$$

Sabendo que $\sigma_{1,2} =  \rho_{1,2}\sigma_1\sigma_2$ é a **covariância** entre o ativo 1 e 2, então a **variância** pode ser dada também por:

$$
\sigma_p^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + 2w_1w_2\sigma_{1,2}
$$

Rearranjando os termos temos:

$$
\sigma_p^2 = w_1^2\sigma_1^2 + 2w_1w_2\sigma_{1,2}  + w_2^2\sigma_2^2 
$$

Que também podemos escrever da forma matricial:

$$
\sigma_p^2 = \begin{pmatrix} w_1  & w_2  \end{pmatrix} 
\begin{pmatrix} \sigma_1^2 & \sigma_{1,2} \\ \sigma_{2,1} & \sigma_2^2  \end{pmatrix}
\begin{pmatrix} w_1 \\ w_2  \end{pmatrix}
$$

$$
\sigma_p^2 = \textbf{w}^T\Sigma\textbf{w}
$$

O retorno do portfólio $R_p$ pode ser dado por:

$$
R_p = \textbf{w}^T\textbf{R}
$$

A volatilidade do portfólio $\sigma_p$ pode ser dada por: 

$$
\sigma_p = \sqrt{ \textbf{w}^T\Sigma\textbf{w}}
$$

Sendo:

$\textbf{w}$: vetor de pesos

$\textbf{w}^T$: veto de pesos transposto


$\textbf{R}$: matriz de retornos

**$\Sigma$**: matriz de covariância